### Install the SDK

In [2]:
%pip install -U -q "google-generativeai>=0.8.3"



sssss

Note: you may need to restart the kernel to use updated packages.


In [3]:
import google.generativeai as genai
from IPython.display import HTML, Markdown, display

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
flash = genai.GenerativeModel('gemini-1.5-flash')
response = flash.generate_content("Explain AI to me like I'm a kid.")
print(response.text)

Imagine you have a super smart friend who can learn really fast. That friend is like AI, or Artificial Intelligence. 

AI is a way for computers to learn and solve problems just like humans do. It's like teaching a computer to think and act smart.

Here's how it works:

* **Learning:** AI learns from lots of information, like pictures, words, or even videos. It's like reading lots of books to learn new things.
* **Thinking:** AI uses what it learned to understand things and make decisions. It's like using your brain to solve a puzzle.
* **Doing:** AI can use its knowledge to do things, like write a story, play a game, or even drive a car!

AI is already helping us in many ways:

* **Siri and Alexa:** These voice assistants use AI to understand what you say and answer your questions.
* **Self-driving cars:** AI helps cars drive safely and avoid accidents.
* **Playing games:** AI can play games against you and even get better over time!

AI is still learning and getting smarter every day

The response often comes back in markdown format, which you can render directly in this notebook.

In [6]:
Markdown(response.text)

Imagine you have a super smart friend who can learn really fast. That friend is like AI, or Artificial Intelligence. 

AI is a way for computers to learn and solve problems just like humans do. It's like teaching a computer to think and act smart.

Here's how it works:

* **Learning:** AI learns from lots of information, like pictures, words, or even videos. It's like reading lots of books to learn new things.
* **Thinking:** AI uses what it learned to understand things and make decisions. It's like using your brain to solve a puzzle.
* **Doing:** AI can use its knowledge to do things, like write a story, play a game, or even drive a car!

AI is already helping us in many ways:

* **Siri and Alexa:** These voice assistants use AI to understand what you say and answer your questions.
* **Self-driving cars:** AI helps cars drive safely and avoid accidents.
* **Playing games:** AI can play games against you and even get better over time!

AI is still learning and getting smarter every day. It's exciting to think about what AI can do for us in the future! 


### Start a chat

The previous example uses a single-turn, text-in/text-out structure, but you can also set up a multi-turn chat structure too.

In [7]:
chat = flash.start_chat(history=[])
response = chat.send_message('Hello! My name is Sathish.')
print(response.text)

Hello Sathish, it's nice to meet you! 👋  What can I do for you today? 😊 



In [8]:
response = chat.send_message('Can you tell something interesting about dinosaurs?')
print(response.text)

Here's something interesting about dinosaurs: 

**Did you know that some dinosaurs had feathers?** 

While we often picture dinosaurs as scaly behemoths, scientists have discovered fossilized evidence of feathers on many dinosaur species, including the iconic Tyrannosaurus Rex. 

This discovery has revolutionized our understanding of these prehistoric creatures, and it suggests that birds are actually the direct descendants of dinosaurs! 

What else would you like to know about dinosaurs? 🤔



In [9]:
# While you have the `chat` object around, the conversation state
# persists. Confirm that by asking if it knows my name.
response = chat.send_message('Do you remember what my name is?')
print(response.text)

Of course!  You are Sathish.  😊  Is there anything else I can help you with? 



### Choose a model

The Gemini API provides access to a number of models from the Gemini model family. Read about the available models and their capabilities on the [model overview page](https://ai.google.dev/gemini-api/docs/models/gemini).

In this step you'll use the API to list all of the available models.

In [10]:
for model in genai.list_models():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/embedding-001
models/text-embedding-004
models/aqa


The [`models.list`](https://ai.google.dev/api/models#method:-models.list) response also returns additional information about the model's capabilities, like the token limits and supported parameters.

In [11]:
for model in genai.list_models():
  if model.name == 'models/gemini-1.5-flash-8b-001':
    print(model)
    break

Model(name='models/gemini-1.5-flash-8b-001',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash-8B 001',
      description='Fast and versatile multimodal model for scaling across diverse tasks',
      input_token_limit=1000000,
      output_token_limit=8192,
      supported_generation_methods=['createCachedContent', 'generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=40)


## Explore generation parameters



### Output length

When generating text with an LLM, the output length affects cost and performance. Generating more tokens increases computation, leading to higher energy consumption, latency, and cost.

To stop the model from generating tokens past a limit, you can specify the `max_output_tokens` parameter when using the Gemini API. Specifying this parameter does not influence the generation of the output tokens, so the output will not become more stylistically or textually succinct, but it will stop generating tokens once the specified length is reached. Prompt engineering may be required to generate a more complete output for your given limit.

In [12]:
short_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(max_output_tokens=200))

response = short_model.generate_content('Write a 1000 word essay on the importance of olives in modern society.')
print(response.text)

## The Enduring Charm of the Olive: A Modern Icon of Health, Culture, and Sustainability

The olive, a seemingly simple fruit, holds a profound significance in modern society, far exceeding its status as a mere culinary ingredient. From ancient times, its versatility and resilience have earned it a place of honor in the culinary, cultural, and economic spheres. Today, the olive continues to thrive, not just as a symbol of Mediterranean life, but as a beacon of healthy eating, cultural heritage, and sustainable practices.

The olive’s journey from ancient times to modern relevance is a testament to its enduring value. In ancient Greece and Rome, olives were considered a symbol of peace, prosperity, and wisdom. Their oil, known for its medicinal properties, was used in religious ceremonies, beauty regimens, and as a staple in daily life. The cultivation of olive trees, requiring patience and dedication, fostered a sense of community and tradition, solidifying their place in cultural narr

In [13]:
response = short_model.generate_content('Write a short poem on the importance of olives in modern society.')
print(response.text)

From ancient groves, a treasure bright,
The olive, with its silver light.
A gift from sun and earth's embrace,
A taste of life in every space.

In salads green, a vibrant hue,
On crusty bread, a salty dew.
In oils so rich, a golden sheen,
A culinary dream, a vibrant scene.

Beyond the plate, a symbol strong,
Of peace and hope, where right belongs.
A timeless tree, its branches wide,
A living link to history's tide.

So raise a glass, let praises ring,
For olive's gift, a precious thing.
In every bite, a story told,
Of life, of love, of stories old. 



Explore with your own prompts. Try a prompt with a restrictive output limit and then adjust the prompt to work within that limit.

### Temperature

Temperature controls the degree of randomness in token selection. Higher temperatures result in a higher number of candidate tokens from which the next output token is selected, and can produce more diverse results, while lower temperatures have the opposite effect, such that a temperature of 0 results in greedy decoding, selecting the most probable token at each step.

Temperature doesn't provide any guarantees of randomness, but it can be used to "nudge" the output somewhat.

In [14]:
from google.api_core import retry

high_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(temperature=2.0))


# When running lots of queries, it's a good practice to use a retry policy so your code
# automatically retries when hitting Resource Exhausted (quota limit) errors.
retry_policy = {
    "retry": retry.Retry(predicate=retry.if_transient_error, initial=10, multiplier=1.5, timeout=300)
}

for _ in range(5):
  response = high_temp_model.generate_content('Pick a random colour... (respond in a single word)',
                                              request_options=retry_policy)
  if response.parts:
    print(response.text, '-' * 25)

Indigo 
 -------------------------
Purple. 
 -------------------------
Purple. 
 -------------------------
Blue 
 -------------------------
Indigo. 
 -------------------------


Now try the same prompt with temperature set to zero. Note that the output is not completely deterministic, as other parameters affect token selection, but the results will tend to be more stable.

In [15]:
low_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(temperature=0.0))

for _ in range(5):
  response = low_temp_model.generate_content('Pick a random colour... (respond in a single word)',
                                             request_options=retry_policy)
  if response.parts:
    print(response.text, '-' * 25)

Purple 
 -------------------------
Purple 
 -------------------------
Purple 
 -------------------------
Purple 
 -------------------------
Purple 
 -------------------------


### Top-K and top-P

Like temperature, top-K and top-P parameters are also used to control the diversity of the model's output.

Top-K is a positive integer that defines the number of most probable tokens from which to select the output token. A top-K of 1 selects a single token, performing greedy decoding.

Top-P defines the probability threshold that, once cumulatively exceeded, tokens stop being selected as candidates. A top-P of 0 is typically equivalent to greedy decoding, and a top-P of 1 typically selects every token in the model's vocabulary.

When both are supplied, the Gemini API will filter top-K tokens first, then top-P and then finally sample from the candidate tokens using the supplied temperature.

Run this example a number of times, change the settings and observe the change in output.

In [16]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        # These are the default values for gemini-1.5-flash-001.
        temperature=1.0,
        top_k=64,
        top_p=0.95,
    ))

story_prompt = "You are a creative writer. Write a short story about a cat who goes on an adventure."
response = model.generate_content(story_prompt, request_options=retry_policy)
print(response.text)

Bartholomew, a ginger tabby with a penchant for mischief, had always been fascinated by the world outside his window. He watched with emerald-green eyes as squirrels chased each other up trees, as birds sang their morning melodies, and as the sun dipped below the horizon, painting the sky in hues of orange and purple. He longed to explore it all, to feel the wind in his whiskers and the earth beneath his paws.

One day, while Mrs. Higgins, his human, was preoccupied with her knitting, Bartholomew saw his chance. He slipped through an open window, landing with a soft thud on the lush, green lawn. He was free!

The world was an explosion of scents and sounds. The sweet smell of honeysuckle filled his nostrils, and the chirping of crickets was a symphony to his ears. He chased a butterfly, its wings a kaleidoscope of color, until it fluttered away, leaving him breathless and exhilarated. 

He encountered a grumpy old tomcat who hissed and growled at him, a flock of sparrows who twittered 

## Prompting

This section contains some prompts from the chapter for you to try out directly in the API. Try changing the text here to see how each prompt performs with different instructions, more examples, or any other changes you can think of.

### Zero-shot

Zero-shot prompts are prompts that describe the request for the model directly.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1gzKKgDHwkAvexG5Up0LMtl1-6jKMKe4g"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [17]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=5,
    ))

zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response = model.generate_content(zero_shot_prompt, request_options=retry_policy)
print(response.text)

Sentiment: **POSITIVE**


#### Enum mode

The models are trained to generate text, and can sometimes produce more text than you may wish for. In the preceding example, the model will output the label, sometimes it can include a preceding "Sentiment" label, and without an output token limit, it may also add explanatory text afterwards.

The Gemini API has an [Enum mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Enum.ipynb) feature that allows you to constrain the output to a fixed set of values.

In [18]:
import enum

class Sentiment(enum.Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"


model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ))

response = model.generate_content(zero_shot_prompt, request_options=retry_policy)
print(response.text)

positive


### One-shot and few-shot

Providing an example of the expected response is known as a "one-shot" prompt. When you provide multiple examples, it is a "few-shot" prompt.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1jjWkjUSoMXmLvMJ7IzADr_GxHPJVV2bg"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>


In [19]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ))

few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "peperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}

ORDER:
"""

customer_order = "Give me a large with cheese & pineapple"


response = model.generate_content([few_shot_prompt, customer_order], request_options=retry_policy)
print(response.text)

```json
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
``` 



#### JSON mode

To provide control over the schema, and to ensure that you only receive JSON (with no other text or markdown), you can use the Gemini API's [JSON mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb). This forces the model to constrain decoding, such that token selection is guided by the supplied schema.

In [20]:
import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str


model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ))

response = model.generate_content("Can I have a large dessert pizza with apple and chocolate")
print(response.text)

{"ingredients": ["apple", "chocolate"], "size": "large", "type": "dessert"}



### Chain of Thought (CoT)

Direct prompting on LLMs can return answers quickly and (in terms of output token usage) efficiently, but they can be prone to hallucination. The answer may "look" correct (in terms of language and syntax) but is incorrect in terms of factuality and reasoning.

Chain-of-Thought prompting is a technique where you instruct the model to output intermediate reasoning steps, and it typically gets better results, especially when combined with few-shot examples. It is worth noting that this technique doesn't completely eliminate hallucinations, and that it tends to cost more to run, due to the increased token count.

As models like the Gemini family are trained to be "chatty" and provide reasoning steps, you can ask the model to be more direct in the prompt.

In [21]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
am 20 years old. How old is my partner? Return the answer immediately."""

model = genai.GenerativeModel('gemini-1.5-flash-latest')
response = model.generate_content(prompt, request_options=retry_policy)

print(response.text)

When you were 4, your partner was 3 * 4 = 12 years old.

This means your partner is 12 - 4 = 8 years older than you.

Therefore, your partner is now 20 + 8 = **28 years old**. 



Now try the same approach, but indicate to the model that it should "think step by step".

In [22]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now,
I am 20 years old. How old is my partner? Let's think step by step."""

response = model.generate_content(prompt, request_options=retry_policy)
print(response.text)

Here's how to solve this problem:

* **When you were 4, your partner was 3 times your age:** 4 years old * 3 = 12 years old.
* **This means your partner is 8 years older than you:** 12 years old - 4 years old = 8 years older.
* **Since you're now 20, your partner is still 8 years older:** 20 years old + 8 years = 28 years old.

**Therefore, your partner is now 28 years old.** 



### ReAct: Reason and act

In this example you will run a ReAct prompt directly in the Gemini API and perform the searching steps yourself. As this prompt follows a well-defined structure, there are frameworks available that wrap the prompt into easier-to-use APIs that make tool calls automatically, such as the LangChain example from the chapter.

To try this out with the Wikipedia search engine, check out the [Searching Wikipedia with ReAct](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb) cookbook example.


> Note: The prompt and in-context examples used here are from [https://github.com/ysymyth/ReAct](https://github.com/ysymyth/ReAct) which is published under a [MIT license](https://opensource.org/licenses/MIT), Copyright (c) 2023 Shunyu Yao.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/18oo63Lwosd-bQ6Ay51uGogB3Wk3H8XMO"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>


In [30]:
model_instructions = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation,
Observation is understanding relevant information from an Action's output and Action can be one of three types:
 (1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it
     will return some similar entities to search and you can try to search the information from those topics.
 (2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches,
     so keep your searches short.
 (3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

example1 = """Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

# Come up with more examples yourself, or take a look through https://github.com/ysymyth/ReAct/

To capture a single step at a time, while ignoring any hallucinated Observation steps, you will use `stop_sequences` to end the generation process. The steps are `Thought`, `Action`, `Observation`, in that order.

In [24]:
question = """Question
Who was the youngest author listed on the transformers NLP paper?
"""

model = genai.GenerativeModel('gemini-1.5-flash-latest')
react_chat = model.start_chat()

# You will perform the Action, so generate up to, but not including, the Observation.
config = genai.GenerationConfig(stop_sequences=["\nObservation"])

resp = react_chat.send_message(
    [model_instructions, example1, example2, question],
    generation_config=config,
    request_options=retry_policy)
print(resp.text)

Thought 1
I need to find the transformers NLP paper and then look for the authors listed. 

Action 1
<search>transformers NLP paper</search>



Now you can perform this research yourself and supply it back to the model.

In [25]:
observation = """Observation 1
[1706.03762] Attention Is All You Need
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.
"""
resp = react_chat.send_message(observation, generation_config=config, request_options=retry_policy)
print(resp.text)

Thought 2: The observation provides the list of authors. I need to determine who is the youngest. 

Action 2: <lookup>youngest</lookup> 



This process repeats until the `<finish>` action is reached. You can continue running this yourself if you like, or try the [Wikipedia example](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb) to see a fully automated ReAct system at work.

## Code prompting

### Generating code

The Gemini family of models can be used to generate code, configuration and scripts. Generating code can be helpful when learning to code, learning a new language or for rapidly generating a first draft.

It's important to be aware that since LLMs can't reason, and can repeat training data, it's essential to read and test your code first, and comply with any relevant licenses.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1YX71JGtzDjXQkgdes8bP6i3oH5lCRKxv"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [26]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=1,
        top_p=1,
        max_output_tokens=1024,
    ))

# Gemini 1.5 models are very chatty, so it helps to specify they stick to the code.
code_prompt = """
Write a Python function to calculate the factorial of a number. No explanation, provide only the code.
"""

response = model.generate_content(code_prompt, request_options=retry_policy)
Markdown(response.text)

```python
def factorial(n):
  if n == 0:
    return 1
  else:
    return n * factorial(n-1)
```

### Code execution

The Gemini API can automatically run generated code too, and will return the output.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/11veFr_VYEwBWcLkhNLr-maCG0G8sS_7Z"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [27]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    tools='code_execution',)

code_exec_prompt = """
Calculate the sum of the first 14 prime numbers. Only consider the odd primes, and make sure you count them all.
"""

response = model.generate_content(code_exec_prompt, request_options=retry_policy)
Markdown(response.text)

I understand.  I will calculate the sum of the first 14 odd prime numbers.


``` python
import sympy

primes = sympy.primerange(0, 100)
odd_primes = [p for p in primes if p % 2 != 0]
sum_of_primes = sum(odd_primes[:14])
print(f'{sum_of_primes=}')

```
```
sum_of_primes=326

```
The sum of the first 14 odd prime numbers is 326. 


While this looks like a single-part response, you can inspect the response to see the each of the steps: initial text, code generation, execution results, and final text summary.

In [28]:
for part in response.candidates[0].content.parts:
  print(part)
  print("-----")

text: "I understand.  I will calculate the sum of the first 14 odd prime numbers.\n\n"

-----
executable_code {
  language: PYTHON
  code: "\nimport sympy\n\nprimes = sympy.primerange(0, 100)\nodd_primes = [p for p in primes if p % 2 != 0]\nsum_of_primes = sum(odd_primes[:14])\nprint(f\'{sum_of_primes=}\')\n"
}

-----
code_execution_result {
  outcome: OUTCOME_OK
  output: "sum_of_primes=326\n"
}

-----
text: "The sum of the first 14 odd prime numbers is 326. \n"

-----


### Explaining code

The Gemini family of models can explain code to you too.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1N7LGzWzCYieyOf_7bAG4plrmkpDNmUyb"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [29]:
file_contents = !curl https://raw.githubusercontent.com/magicmonty/bash-git-prompt/refs/heads/master/gitprompt.sh

explain_prompt = f"""
Please explain what this file does at a very high level. What is it, and why would I use it?

```
{file_contents}
```
"""

model = genai.GenerativeModel('gemini-1.5-flash-latest')

response = model.generate_content(explain_prompt, request_options=retry_policy)
Markdown(response.text)

This file is a Bash/Zsh script called `git-prompt.sh` that provides a customized Git prompt for your terminal. It displays information about the current Git repository, branch, status, and other relevant details, making it easier to navigate and work with Git projects.

Here's why you might use it:

* **Enhanced Git awareness:** The script provides a rich and informative Git prompt that helps you quickly understand the state of your Git repository, such as the current branch, whether you have uncommitted changes, and if you're ahead or behind the remote branch.
* **Improved workflow:** The script streamlines your Git workflow by providing quick access to essential information without having to run separate Git commands.
* **Customization options:** The script offers a wide range of customization options, allowing you to tailor the prompt to your preferences, including colors, symbols, and the information displayed.

The file contains functions to:

* **Load colors:**  Defines color schemes for the prompt.
* **Load themes:**  Allows you to select different themes for the prompt's appearance.
* **Configure Git prompt:** Sets up various settings for the prompt, including remote status checking, display options, and prompt customization.
* **Update the prompt:**  Updates the prompt based on the current Git repository status, branch, and other relevant information.
* **Install the prompt:**  Integrates the script with your shell's prompt mechanism to display the customized Git prompt.

Overall, `git-prompt.sh` enhances your Git experience by providing a visually appealing and informative prompt that keeps you informed about your repository's status and facilitates smoother navigation.


## Learn more

To learn more about prompting in depth:

* Check out the whitepaper issued with today's content,
* Try out the apps listed at the top of this notebook ([TextFX](https://textfx.withgoogle.com/), [SQL Talk](https://sql-talk-r5gdynozbq-uc.a.run.app/) and [NotebookLM](https://notebooklm.google/)),
* Read the [Introduction to Prompting](https://ai.google.dev/gemini-api/docs/prompting-intro) from the Gemini API docs,
* Explore the Gemini API's [prompt gallery](https://ai.google.dev/gemini-api/prompts) and try them out in AI Studio,
* Check out the Gemini API cookbook for [inspirational examples](https://github.com/google-gemini/cookbook/blob/main/examples/) and [educational quickstarts](https://github.com/google-gemini/cookbook/blob/main/quickstarts/).

And please share anything exciting you have tried in the Discord!